<a href="https://colab.research.google.com/github/TheDarKnight50/SOC---LLM/blob/main/Lectures/Lec_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-06-08 10:06:56--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.01s   

2024-06-08 10:06:56 (16.3 MB/s) - ‘names.txt’ saved [228145/228145]



In [4]:
words = open('/content/names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
len(words)

32033

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s : i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i : s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
# Building the dataset

block_size = 3
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size # Build a list corresponding to : [0, 0, 0, ...]

  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]  # Crop from first letter and slide

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [8]:
C = torch.randn(27, 2)

In [10]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [11]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [32]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[-0.9074,  0.8466, -0.9747,  ...,  0.1535,  0.9808, -0.4393],
        [ 0.1059, -0.9677, -0.8811,  ...,  0.7618,  0.9999,  0.6302],
        [ 0.3984, -0.9718, -0.6606,  ..., -0.0432, -0.4956, -1.0000],
        ...,
        [ 0.2196, -0.8149, -0.8242,  ..., -0.8827,  0.2664, -0.9946],
        [-0.9973, -0.8884, -0.8757,  ..., -0.9970,  0.9853,  0.7050],
        [-0.9836, -0.9905, -0.7034,  ..., -0.8189,  0.9913, -0.8745]])

In [35]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [42]:
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdims = True)
prob.shape

torch.Size([32, 27])

In [44]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.0179)

In [45]:
# -------- summary of above : ----------

In [47]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator = g)

W1 = torch.randn((6, 100), generator = g)
b1 = torch.randn(100, generator = g)

W2 = torch.randn((100, 27), generator = g)
b2 = torch.randn(27, generator = g)

parameters = [C, W1, b1, W2 ,b2]

In [50]:
sum(p.nelement() for p in parameters)

3481

In [56]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

logits = h @ W2 + b2
#counts = logits.exp()
#prob = counts / counts.sum(1, keepdims = True)
#prob.shape

loss = F.cross_entropy(logits, Y) # Inbuilt function to do the same thing :)
loss

tensor(17.7697)